In [1]:
import gpflow as gp
import numpy as np
from bayes_tec.datapack import DataPack
from bayes_tec.utils.data_utils import make_coord_array
import pymc3 as pm

/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/multipledispatch-0.4.9-py3.6.egg/multipledispatch/dispatcher.py:24: AmbiguityWarning: 
Ambiguities exist in dispatched function _expectation

The following signatures may result in ambiguous behavior:
	[Gaussian, Linear, NoneType, Sum, InducingPoints], [Gaussian, Identity, NoneType, Kernel, InducingPoints]


Consider making the following additions:

@dispatch(Gaussian, Identity, NoneType, Sum, InducingPoints)
def _expectation(...)
  warn(warning_text(dispatcher.name, ambiguities), AmbiguityWarning)


## Generate Data

In [2]:
with DataPack('../../scripts/data/killms_datapack_2.hdf5',readonly = True) as datapack:
    datapack.switch_solset('sol000')
    datapack.select(ant='RS210HBA',time=slice(50,55,1),pol=slice(0,1,1),freq=slice(0,48,20))
    phase,axes = datapack.phase
    patch_names, directions = datapack.get_sources(axes['dir'])
    antenna_labels, antenans = datapack.get_antennas(axes['ant'])
    _, freqs = datapack.get_freqs(axes['freq'])
    _, times = datapack.get_times(axes['time'])
    _, pols = datapack.get_pols(axes['pol'])
    
X_t = (times.mjd*86400. - times[0].mjd*86400.)[:,None]
X_t = (X_t - X_t.mean())/X_t.std()
X_d = np.array([directions.ra.deg-directions.ra.deg.mean(), directions.dec.deg-directions.dec.deg.mean()]).T
X_d /= X_d.std()

X_f = freqs[:,None]

X = make_coord_array(X_d, X_f, X_t,flat=True)
X_freqs = X[:,[2]]
X = X[:,[0,1,3]]

Y = phase[0,:,0,:,:].reshape((-1,1))



In [3]:
import theano.tensor as tt

In [5]:
with pm.Model() as model:
    tec_scale = 0.005
    X_ = tt.constant(X.copy())
    Y_ = tt.constant(Y.copy())
    X_freqs_ = tt.constant(X_freqs.copy())
    sigma = pm.Exponential('sigma',1.)
    ls = pm.Uniform('ls',0.,2.,shape=3)
    cov_func = sigma**2 * pm.gp.cov.ExpQuad(input_dim=3, ls=ls)
    mean_func = pm.gp.mean.Zero()
    gp = pm.gp.Latent(mean_func, cov_func)
    f = -tec_scale*8.448e9*gp.prior('f',X)/X_freqs_
    sigma = pm.HalfNormal('sigma_D',0.25*np.pi)
    vM = pm.VonMises('likelihood',mu=f, kappa=1./sigma**2,observed=Y_)
    trace = pm.sample(1000,use_mmap=False)
    

Process ForkPoolWorker-9:
Traceback (most recent call last):
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/joblib-0.11-py3.6.egg/joblib/pool.py", line 362, in get
    return recv()
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/theano/compile/function_module.py", line 1082, in _constructor_Function
    f = maker.cre

  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/theano/gof/link.py", line 699, in make_thunk
    storage_map=storage_map)[:3]
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/theano/gof/vm.py", line 1084, in make_all
    impl=impl))
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/theano/gof/op.py", line 833, in make_c_thunk
    e = FunctionGraph(node.inputs, node.outputs)
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/theano/gof/fg.py", line 137, in __init__
    inputs, outputs = graph.clone(inputs, outputs)
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/theano/gof/graph.py", line 862, in clone
    equiv = clone_get_equiv(i, o, copy_inputs, copy_orphans)
  Fil

  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/joblib-0.11-py3.6.egg/joblib/pool.py", line 360, in get
    racquire()
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

## Make wrapped phase model homoscedastic GPR

In [ ]:
import tensorflow as tf

from gpflow import likelihoods
from gpflow import settings

from gpflow.conditionals import base_conditional
from gpflow.params import DataHolder
from gpflow.decors import params_as_tensors
from gpflow.decors import name_scope
from gpflow.logdensities import multivariate_normal

from gpflow.models.model import GPModel

try:
    @tf.RegisterGradient('WrapGrad')
    def _wrap_grad(op,grad):
        phi = op.inputs[0]
        return tf.ones_like(phi)*grad

    def wrap(phi):
        out = tf.atan2(tf.sin(phi),tf.cos(phi))
        with tf.get_default_graph().gradient_override_map({'Identity': 'WrapGrad'}):
            return tf.identity(out)

except:
    pass#already defined

class WrappedGPR(GPModel):
    """
    Gaussian Process Regression.
    This is a vanilla implementation of GP regression with a Gaussian
    likelihood.  Multiple columns of Y are treated independently.
    The log likelihood i this models is sometimes referred to as the 'marginal log likelihood', and is given by
    .. math::
       \\log p(\\mathbf y \\,|\\, \\mathbf f) = \\mathcal N\\left(\\mathbf y\,|\, 0, \\mathbf K + \\sigma_n \\mathbf I\\right)
    """
    def __init__(self, X, Y, Y_freqs, kern, mean_function=None, **kwargs):
        """
        X is a data matrix, size N x D
        Y is a data matrix, size N x R
        kern, mean_function are appropriate GPflow objects
        """
        likelihood = likelihoods.Gaussian()
        X = DataHolder(X)
        Y = DataHolder(Y)
        GPModel.__init__(self, X, Y, kern, likelihood, mean_function, **kwargs)
        self.Y_freqs = DataHolder(Y_freqs)

    @name_scope('likelihood')
    @params_as_tensors
    def _build_likelihood(self):
        """
        Construct a tensorflow function to compute the likelihood.
            \log p(Y | theta).
        """
        K = self.kern.K(self.X) + tf.eye(tf.shape(self.X)[0], dtype=settings.float_type) * self.likelihood.variance
        L = tf.cholesky(K)
        m = self.mean_function(self.X)
        logpdf = multivariate_normal(wrap(wrap(self.Y) - wrap(m)), 0.*m, L)  # (R,) log-likelihoods for each independent dimension of Y

        return tf.reduce_sum(logpdf)

    @name_scope('predict')
    @params_as_tensors
    def _build_predict(self, Xnew, full_cov=False):
        """
        Xnew is a data matrix, point at which we want to predict
        This method computes
            p(F* | Y )
        where F* are points on the GP at Xnew, Y are noisy observations at X.
        """
        y = self.Y - self.mean_function(self.X)
        Kmn = self.kern.K(self.X, Xnew)
        Kmm_sigma = self.kern.K(self.X) + tf.eye(tf.shape(self.X)[0], dtype=settings.float_type) * self.likelihood.variance
        Knn = self.kern.K(Xnew) if full_cov else self.kern.Kdiag(Xnew)
        f_mean, f_var = base_conditional(Kmn, Kmm_sigma, Knn, y, full_cov=full_cov, white=False)  # N x P, N x P or P x N x N
        return f_mean + self.mean_function(Xnew), f_var